# Quantitative Momentum Strategy

"Momentum investing" means investing in the stocks that have increased in price the most.

For this project, we're going to build an investing strategy that selects the 50 stocks with the highest price momentum. From there, we will calculate recommended trades for an equal-weight portfolio of these 50 stocks.


## Library Imports

The first thing we need to do is import the open-source software libraries that we'll be using in this tutorial.

In [96]:
import numpy as np
import pandas as pd
import requests
import math
from scipy import stats
import xlsxwriter

## Importing Our List of Stocks

As before, we'll need to import our list of stocks and our API token before proceeding. Make sure the `.csv` file is still in your working directory and import it with the following command:

In [97]:
stocks  = pd.read_csv('sp_500_stocks.csv')
from secrets import IEX_CLOUD_API_TOKEN

## Making Our First API Call

It's now time to make the first version of our momentum screener!

We need to get one-year price returns for each stock in the universe. Here's how.

In [98]:
symbol = 'AAPL'
api_url = f'https://sandbox.iexapis.com/stable/stock/{symbol}/stats?token={IEX_CLOUD_API_TOKEN}'
data = requests.get(api_url).json()
data


{'companyName': 'Apple Inc',
 'marketcap': 2859136589303,
 'week52high': 185.28,
 'week52low': 118.32,
 'week52highSplitAdjustOnly': 183.2,
 'week52lowSplitAdjustOnly': 116.64,
 'week52change': 0.2719309305779872,
 'sharesOutstanding': 16649797430,
 'float': 0,
 'avg10Volume': 110510235,
 'avg30Volume': 107231141,
 'day200MovingAvg': 163.41,
 'day50MovingAvg': 173.71,
 'employees': 150548,
 'ttmEPS': 6.18,
 'ttmDividendRate': 0.9136487582569521,
 'dividendYield': 0.00531549970860832,
 'nextDividendDate': '',
 'exDividendDate': '2022-02-03',
 'nextEarningsDate': '2022-04-21',
 'peRatio': 28.11093584108121,
 'beta': 1.3703995911585067,
 'maxChangePercent': 66.38403814655366,
 'year5ChangePercent': 4.829008019592847,
 'year2ChangePercent': 1.254496438094739,
 'year1ChangePercent': 0.2675292283653689,
 'ytdChangePercent': -0.028301844567987,
 'month6ChangePercent': 0.1824850228278269,
 'month3ChangePercent': 0.14961827589971913,
 'month1ChangePercent': -0.04058661995524445,
 'day30ChangePe

## Parsing Our API Call

This API call has all the information we need. We can parse it using the same square-bracket notation as in the first project of this course. Here is an example.

In [99]:
data['year1ChangePercent']

0.2675292283653689

## Executing A Batch API Call & Building Our DataFrame

Just like in our first project, it's now time to execute several batch API calls and add the information we need to our DataFrame.

We'll start by running the following code cell, which contains some code we already built last time that we can re-use for this project. More specifically, it contains a function called `chunks` that we can use to divide our list of securities into groups of 100.

In [100]:
# Function sourced from 
# https://stackoverflow.com/questions/312443/how-do-you-split-a-list-into-evenly-sized-chunks
def chunks(lst, n):
    """Yield successive n-sized chunks from lst."""
    for i in range(0, len(lst), n):
        yield lst[i:i + n]   
        
symbol_groups = list(chunks(stocks['Ticker'], 100))
symbol_strings = []
for i in range(0, len(symbol_groups)):
    symbol_strings.append(','.join(symbol_groups[i]))
#     print(symbol_strings[i])

my_columns = ['Ticker', 'Price', 'One-Year Price Return', 'Number of Shares to Buy']

Now we need to create a blank DataFrame and add our data to the data frame one-by-one.

In [101]:
final_dataframe = pd.DataFrame(columns=my_columns)

for symbol_string in symbol_strings:
    batch_api_call_url = f'https://sandbox.iexapis.com/stable/stock/market/batch/?types=stats,price&symbols={symbol_string}&token={IEX_CLOUD_API_TOKEN}'
    data = requests.get(batch_api_call_url).json()
    for symbol in symbol_string.split(','):
        final_dataframe = final_dataframe.append(
            pd.Series(
                [
                    symbol,
                    data[symbol]['price'],
                    data[symbol]['stats']['year1ChangePercent'],
                    'N/A'
                ],
                index = my_columns
            ),
            ignore_index = True
        )
final_dataframe

,Ticker,Price,One-Year Price Return,Number of Shares to Buy
0,A,143.18,0.149578,N/A
1,AAL,16.97,-0.055914,N/A
2,AAP,227.37,0.459801,N/A
3,AAPL,180.23,0.266362,N/A
4,ABBV,144.89,0.366323,N/A
...,...,...,...,...
500,YUM,126.33,0.240938,N/A
501,ZBH,124.78,-0.238708,N/A
502,ZBRA,524.52,0.243584,N/A
503,ZION,71.57,0.51664,N/A


## Removing Low-Momentum Stocks

The investment strategy that we're building seeks to identify the 50 highest-momentum stocks in the S&P 500.

Because of this, the next thing we need to do is remove all the stocks in our DataFrame that fall below this momentum threshold. We'll sort the DataFrame by the stocks' one-year price return, and drop all stocks outside the top 50.


In [102]:
final_dataframe.sort_values('One-Year Price Return', ascending=False, inplace=True)
final_dataframe=final_dataframe[:50]
final_dataframe.reset_index(inplace=True)
final_dataframe


,index,Ticker,Price,One-Year Price Return,Number of Shares to Buy
0,275,LB,80.98,2.349313,N/A
1,148,DVN,53.61,2.153039,N/A
2,317,MRO,22.30,1.848368,N/A
3,344,NUE,111.41,1.204466,N/A
4,109,COP,92.34,1.196949,N/A
5,160,EOG,117.90,1.109844,N/A
6,175,FANG,135.65,1.10152,N/A
7,299,MCHP,74.83,1.077721,N/A
8,195,FTNT,319.97,1.050718,N/A
9,386,PXD,235.33,0.892533,N/A


## Calculating the Number of Shares to Buy

Just like in the last project, we now need to calculate the number of shares we need to buy. The one change we're going to make is wrapping this functionality inside a function, since we'll be using it again later in this Jupyter Notebook.

Since we've already done most of the work on this, try to complete the following two code cells without watching me do it first!

In [103]:
def portfolio_input():
    global portfolio_size
    portfolio_size = input("Enter the value of your portfolio:")

    try:
        val = float(portfolio_size)
    except ValueError:
        print("That's not a number! \n Try again:")
        portfolio_size = input("Enter the value of your portfolio:")

portfolio_input()
print(portfolio_size)

100000


In [104]:
position_size = float(portfolio_size)/len(final_dataframe.index)
for i in range(0, len(final_dataframe)):
    final_dataframe.loc[i, 'Number of Shares to Buy'] = math.floor(position_size/final_dataframe.loc[i, 'Price'])

final_dataframe

,index,Ticker,Price,One-Year Price Return,Number of Shares to Buy
0,275,LB,80.98,2.349313,24
1,148,DVN,53.61,2.153039,37
2,317,MRO,22.30,1.848368,89
3,344,NUE,111.41,1.204466,17
4,109,COP,92.34,1.196949,21
5,160,EOG,117.90,1.109844,16
6,175,FANG,135.65,1.10152,14
7,299,MCHP,74.83,1.077721,26
8,195,FTNT,319.97,1.050718,6
9,386,PXD,235.33,0.892533,8


## Building a Better (and More Realistic) Momentum Strategy

Real-world quantitative investment firms differentiate between "high quality" and "low quality" momentum stocks:

* High-quality momentum stocks show "slow and steady" outperformance over long periods of time
* Low-quality momentum stocks might not show any momentum for a long time, and then surge upwards.

The reason why high-quality momentum stocks are preferred is because low-quality momentum can often be cause by short-term news that is unlikely to be repeated in the future (such as an FDA approval for a biotechnology company).

To identify high-quality momentum, we're going to build a strategy that selects stocks from the highest percentiles of: 

* 1-month price returns
* 3-month price returns
* 6-month price returns
* 1-year price returns

Let's start by building our DataFrame. You'll notice that I use the abbreviation `hqm` often. It stands for `high-quality momentum`.

In [105]:
hqm_columns = [
                'Ticker', 
                'Price', 
                'Number of Shares to Buy', 
                'One-Year Price Return', 
                'One-Year Return Percentile',
                'Six-Month Price Return',
                'Six-Month Return Percentile',
                'Three-Month Price Return',
                'Three-Month Return Percentile',
                'One-Month Price Return',
                'One-Month Return Percentile',
                'HQM Score'
                ]

hqm_dataframe = pd.DataFrame(columns = hqm_columns)

for symbol_string in symbol_strings:
    batch_api_call_url = f'https://sandbox.iexapis.com/stable/stock/market/batch/?types=stats,price,quote&symbols={symbol_string}&token={IEX_CLOUD_API_TOKEN}'
    data = requests.get(batch_api_call_url).json()
    for symbol in symbol_string.split(','):
        hqm_dataframe.dropna(inplace=True)
        hqm_dataframe = hqm_dataframe.append(
            pd.Series(
                [
                    symbol,
                    data[symbol]['quote']['latestPrice'],
                    'N/A',
                    data[symbol]['stats']['year1ChangePercent'],
                    'N/A',
                    data[symbol]['stats']['month6ChangePercent'],
                    'N/A',
                    data[symbol]['stats']['month3ChangePercent'],
                    'N/A',
                    data[symbol]['stats']['month1ChangePercent'],
                    'N/A',
                    'N/A'
                ],
                index = hqm_columns
            ),
            ignore_index = True
        )  
hqm_dataframe

,Ticker,Price,Number of Shares to Buy,One-Year Price Return,One-Year Return Percentile,Six-Month Price Return,Six-Month Return Percentile,Three-Month Price Return,Three-Month Return Percentile,One-Month Price Return,One-Month Return Percentile,HQM Score
0,A,145.09,N/A,0.148235,N/A,-0.085504,N/A,-0.104663,N/A,-0.068076,N/A,N/A
1,AAL,16.85,N/A,-0.056475,N/A,-0.158031,N/A,-0.210261,N/A,-0.141632,N/A,N/A
2,AAP,233.46,N/A,0.474981,N/A,0.096355,N/A,-0.009462,N/A,-0.044149,N/A,N/A
3,AAPL,172.42,N/A,0.273907,N/A,0.180058,N/A,0.148857,N/A,-0.039628,N/A,N/A
4,ABBV,145.58,N/A,0.358426,N/A,0.250219,N/A,0.221595,N/A,0.051612,N/A,N/A
...,...,...,...,...,...,...,...,...,...,...,...,...
496,YUM,127.79,N/A,0.247077,N/A,-0.055172,N/A,0.003027,N/A,-0.0908,N/A,N/A
497,ZBH,126.62,N/A,-0.233272,N/A,-0.177013,N/A,-0.112986,N/A,-0.059684,N/A,N/A
498,ZBRA,514.33,N/A,0.24674,N/A,-0.105401,N/A,-0.16173,N/A,-0.150249,N/A,N/A
499,ZION,71.83,N/A,0.538033,N/A,0.391613,N/A,0.115712,N/A,0.079812,N/A,N/A


## Calculating Momentum Percentiles

We now need to calculate momentum percentile scores for every stock in the universe. More specifically, we need to calculate percentile scores for the following metrics for every stock:

* `One-Year Price Return`
* `Six-Month Price Return`
* `Three-Month Price Return`
* `One-Month Price Return`

Here's how we'll do this:

In [109]:
time_periods = [
                'One-Year',
                'Six-Month',
                'Three-Month',
                'One-Month'
                ]
for row in hqm_dataframe.index:
    for time_period in  time_periods:
        change_col = f'{time_period} Price Return'
        percentile_col = f'{time_period} Return Percentile'
        hqm_dataframe.loc[row, percentile_col] = stats.percentileofscore(hqm_dataframe[change_col], hqm_dataframe.loc[row, change_col])/100
 
hqm_dataframe

,Ticker,Price,Number of Shares to Buy,One-Year Price Return,One-Year Return Percentile,Six-Month Price Return,Six-Month Return Percentile,Three-Month Price Return,Three-Month Return Percentile,One-Month Price Return,One-Month Return Percentile,HQM Score
0,A,145.09,N/A,0.148235,0.45509,-0.085504,0.225549,-0.104663,0.225549,-0.068076,0.443114,33.732535
1,AAL,16.85,N/A,-0.056475,0.135729,-0.158031,0.10978,-0.210261,0.0499,-0.141632,0.131737,10.678643
2,AAP,233.46,N/A,0.474981,0.864271,0.096355,0.682635,-0.009462,0.52495,-0.044149,0.516966,64.720559
3,AAPL,172.42,N/A,0.273907,0.646707,0.180058,0.818363,0.148857,0.91018,-0.039628,0.540918,72.904192
4,ABBV,145.58,N/A,0.358426,0.778443,0.250219,0.898204,0.221595,0.97006,0.051612,0.870259,87.924152
...,...,...,...,...,...,...,...,...,...,...,...,...
496,YUM,127.79,N/A,0.247077,0.612774,-0.055172,0.297405,0.003027,0.56487,-0.0908,0.327345,45.05988
497,ZBH,126.62,N/A,-0.233272,0.027944,-0.177013,0.095808,-0.112986,0.197605,-0.059684,0.477046,19.96008
498,ZBRA,514.33,N/A,0.24674,0.610778,-0.105401,0.193613,-0.16173,0.101796,-0.150249,0.10978,25.399202
499,ZION,71.83,N/A,0.538033,0.92016,0.391613,0.962076,0.115712,0.850299,0.079812,0.928144,91.516966


## Calculating the HQM Score

We'll now calculate our `HQM Score`, which is the high-quality momentum score that we'll use to filter for stocks in this investing strategy.

The `HQM Score` will be the arithmetic mean of the 4 momentum percentile scores that we calculated in the last section.

To calculate arithmetic mean, we will use the `mean` function from Python's built-in `statistics` module.

In [110]:
from statistics import mean 

for row in hqm_dataframe.index:
    momentum_percentiles = []
    for time_period in time_periods:
        momentum_percentiles.append(hqm_dataframe.loc[row, f'{time_period} Return Percentile'])
    hqm_dataframe.loc[row, 'HQM Score'] = mean(momentum_percentiles)

hqm_dataframe
    

,Ticker,Price,Number of Shares to Buy,One-Year Price Return,One-Year Return Percentile,Six-Month Price Return,Six-Month Return Percentile,Three-Month Price Return,Three-Month Return Percentile,One-Month Price Return,One-Month Return Percentile,HQM Score
0,A,145.09,N/A,0.148235,0.45509,-0.085504,0.225549,-0.104663,0.225549,-0.068076,0.443114,0.337325
1,AAL,16.85,N/A,-0.056475,0.135729,-0.158031,0.10978,-0.210261,0.0499,-0.141632,0.131737,0.106786
2,AAP,233.46,N/A,0.474981,0.864271,0.096355,0.682635,-0.009462,0.52495,-0.044149,0.516966,0.647206
3,AAPL,172.42,N/A,0.273907,0.646707,0.180058,0.818363,0.148857,0.91018,-0.039628,0.540918,0.729042
4,ABBV,145.58,N/A,0.358426,0.778443,0.250219,0.898204,0.221595,0.97006,0.051612,0.870259,0.879242
...,...,...,...,...,...,...,...,...,...,...,...,...
496,YUM,127.79,N/A,0.247077,0.612774,-0.055172,0.297405,0.003027,0.56487,-0.0908,0.327345,0.450599
497,ZBH,126.62,N/A,-0.233272,0.027944,-0.177013,0.095808,-0.112986,0.197605,-0.059684,0.477046,0.199601
498,ZBRA,514.33,N/A,0.24674,0.610778,-0.105401,0.193613,-0.16173,0.101796,-0.150249,0.10978,0.253992
499,ZION,71.83,N/A,0.538033,0.92016,0.391613,0.962076,0.115712,0.850299,0.079812,0.928144,0.91517


## Selecting the 50 Best Momentum Stocks

As before, we can identify the 50 best momentum stocks in our universe by sorting the DataFrame on the `HQM Score` column and dropping all but the top 50 entries.

In [111]:
hqm_dataframe.sort_values('HQM Score', ascending=False, inplace=True)
hqm_dataframe = hqm_dataframe[:50]
hqm_dataframe.reset_index(drop=True, inplace=True)
hqm_dataframe

,Ticker,Price,Number of Shares to Buy,One-Year Price Return,One-Year Return Percentile,Six-Month Price Return,Six-Month Return Percentile,Three-Month Price Return,Three-Month Return Percentile,One-Month Price Return,One-Month Return Percentile,HQM Score
0,MRO,21.50,N/A,1.795438,0.996008,0.978368,0.994012,0.305479,0.996008,0.229298,0.992016,0.994511
1,EOG,116.70,N/A,1.108429,0.988024,0.681109,0.982036,0.303103,0.994012,0.210871,0.99002,0.988523
2,COP,94.14,N/A,1.196025,0.994012,0.700011,0.988024,0.264371,0.984032,0.196842,0.986028,0.988024
3,DVN,55.07,N/A,2.087799,0.998004,1.205378,1.0,0.274098,0.988024,0.114246,0.962076,0.987026
4,HAL,32.79,N/A,0.689958,0.956088,0.686688,0.984032,0.311437,0.998004,0.263135,1.0,0.984531
5,XOM,84.02,N/A,0.797952,0.972056,0.493537,0.976048,0.286422,0.99002,0.237257,0.996008,0.983533
6,PXD,234.84,N/A,0.878322,0.982036,0.593732,0.978044,0.255929,0.978044,0.192721,0.984032,0.980539
7,OXY,42.40,N/A,0.839086,0.98004,0.59993,0.98004,0.215411,0.964072,0.233063,0.994012,0.979541
8,APA,34.71,N/A,0.811431,0.974052,1.002642,0.996008,0.199733,0.958084,0.175828,0.978044,0.976547
9,LB,83.01,N/A,2.311379,1.0,0.834647,0.992016,0.21706,0.968064,0.080524,0.93014,0.972555


## Calculating the Number of Shares to Buy

We'll use the `portfolio_input` function that we created earlier to accept our portfolio size. Then we will use similar logic in a `for` loop to calculate the number of shares to buy for each stock in our investment universe.

In [112]:
portfolio_input()

In [113]:
position_size = float(portfolio_size)/len(hqm_dataframe.index)
for i in hqm_dataframe.index:
    hqm_dataframe.loc[i, 'Number of Shares to Buy'] = math.floor(position_size/hqm_dataframe.loc[i, 'Price'])

hqm_dataframe

,Ticker,Price,Number of Shares to Buy,One-Year Price Return,One-Year Return Percentile,Six-Month Price Return,Six-Month Return Percentile,Three-Month Price Return,Three-Month Return Percentile,One-Month Price Return,One-Month Return Percentile,HQM Score
0,MRO,21.50,9,1.795438,0.996008,0.978368,0.994012,0.305479,0.996008,0.229298,0.992016,0.994511
1,EOG,116.70,1,1.108429,0.988024,0.681109,0.982036,0.303103,0.994012,0.210871,0.99002,0.988523
2,COP,94.14,2,1.196025,0.994012,0.700011,0.988024,0.264371,0.984032,0.196842,0.986028,0.988024
3,DVN,55.07,3,2.087799,0.998004,1.205378,1.0,0.274098,0.988024,0.114246,0.962076,0.987026
4,HAL,32.79,6,0.689958,0.956088,0.686688,0.984032,0.311437,0.998004,0.263135,1.0,0.984531
5,XOM,84.02,2,0.797952,0.972056,0.493537,0.976048,0.286422,0.99002,0.237257,0.996008,0.983533
6,PXD,234.84,0,0.878322,0.982036,0.593732,0.978044,0.255929,0.978044,0.192721,0.984032,0.980539
7,OXY,42.40,4,0.839086,0.98004,0.59993,0.98004,0.215411,0.964072,0.233063,0.994012,0.979541
8,APA,34.71,5,0.811431,0.974052,1.002642,0.996008,0.199733,0.958084,0.175828,0.978044,0.976547
9,LB,83.01,2,2.311379,1.0,0.834647,0.992016,0.21706,0.968064,0.080524,0.93014,0.972555


## Formatting Our Excel Output

We will be using the XlsxWriter library for Python to create nicely-formatted Excel files.

XlsxWriter is an excellent package and offers tons of customization. However, the tradeoff for this is that the library can seem very complicated to new users. Accordingly, this section will be fairly long because I want to do a good job of explaining how XlsxWriter works.

In [114]:
writer = pd.ExcelWriter('momentum_strategy.xlsx', engine='xlsxwriter')
hqm_dataframe.to_excel(writer, sheet_name = 'Momentum Strategy', index=False)

## Creating the Formats We'll Need For Our .xlsx File

You'll recall from our first project that formats include colors, fonts, and also symbols like % and $. We'll need four main formats for our Excel document:

* String format for tickers
* \$XX.XX format for stock prices
* \$XX,XXX format for market capitalization
* Integer format for the number of shares to purchase

Since we already built our formats in the last section of this course, I've included them below for you. Run this code cell before proceeding.

In [115]:
background_color = '#0a0a23'
font_color = '#ffffff'

string_template = writer.book.add_format(
        {
            'font_color': font_color,
            'bg_color': background_color,
            'border': 1
        }
    )

dollar_template = writer.book.add_format(
        {
            'num_format':'$0.00',
            'font_color': font_color,
            'bg_color': background_color,
            'border': 1
        }
    )

integer_template = writer.book.add_format(
        {
            'num_format':'0',
            'font_color': font_color,
            'bg_color': background_color,
            'border': 1
        }
    )

percent_template = writer.book.add_format(
        {
            'num_format':'0.0%',
            'font_color': font_color,
            'bg_color': background_color,
            'border': 1
        }
    )

In [116]:
column_formats = { 
                    'A': ['Ticker', string_template],
                    'B': ['Price', dollar_template],
                    'C': ['Number of Shares to Buy', integer_template],
                    'D': ['One-Year Price Return', percent_template],
                    'E': ['One-Year Return Percentile', percent_template],
                    'F': ['Six-Month Price Return', percent_template],
                    'G': ['Six-Month Return Percentile', percent_template],
                    'H': ['Three-Month Price Return', percent_template],
                    'I': ['Three-Month Return Percentile', percent_template],
                    'J': ['One-Month Price Return', percent_template],
                    'K': ['One-Month Return Percentile', percent_template],
                    'L': ['HQM Score', integer_template]
                    }

for column in column_formats.keys():
    writer.sheets['Momentum Strategy'].set_column(f'{column}:{column}', 20, column_formats[column][1])
    writer.sheets['Momentum Strategy'].write(f'{column}1', column_formats[column][0], string_template)

## Saving Our Excel Output

As before, saving our Excel output is very easy:

In [117]:
writer.save()